<a href="https://colab.research.google.com/github/solee328/LikeLion_EDA/blob/test/EDA_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>